# Sentiment analysis of show reviews

The goal of this analysis is to ...
* Get hands-on experience with packages and tools for analysing Russian language (natasha, nltk, spacy, rnnmorph, pymorphy2)
* Investigate available pre-trained models for Russian language (wor2vec, fasttext, navec, models from sber, deeppavlov and others)
* Learn how to finetune BERT-like models

## Imports

In [19]:
import os
import sys
import warnings

import dateparser
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

tqdm.pandas()

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [4]:
SEED = 42

## Data

### Loading data

In [2]:
reviews_df = pd.read_parquet("../../data/mt_reviews.parquet")
reviews_df.shape

(206737, 9)

### Dataset overview

In [5]:
reviews_df.sample(n=10, random_state=SEED)

,show_id,user_id,type,datetime,sentiment,subtitle,review_body,usefulness,scores
196236,257386,28525,movie,2010-08-27 11:23:00,good,Вечный город.,К истории Древнего мира у меня отношение особо...,40,
128582,688832,44953,movie,2015-02-20 23:19:00,bad,50 оттенков разочарования,"Говорю сразу, книги читала все, да и по нескол...",5,3 из 10
159673,349,33910,movie,2018-04-15 21:04:00,good,"Господи, спасибо, что не пронесло мимо","Есть два типа фильмов, мой друг. Одни ты прост...",4,
109244,686898,44065,movie,2019-11-09 11:58:00,neutral,Что же стало с клоуном?,"Итак, в первую очередь хотелось бы отметить то...",2,"6,5 из 10"
92610,61455,66782,movie,2017-11-27 18:52:00,good,Они отказываются подчиняться,"Автора этого замечательного фильма, Джосса Уэд...",5,
43727,491724,44563,movie,2012-01-28 23:18:00,good,Жестокая правда,"Финчер снова нас поразил, он всегда нас поража...",7,10 из 10
50195,102130,67145,movie,2009-08-17 11:56:00,good,"Преодолеть 2 года жизни, что встретиться","Слышал о фильме много, и в основном положитель...",11,10 из 10
43651,491724,66283,movie,2012-02-19 15:33:00,bad,"Мужчины, которые ненавидели женщин.",Я попробовала рассматривать этот фильм с двух ...,3,6 из 10
48775,7226,7905,movie,2014-02-20 03:44:00,good,None,"«Догвилль» - это один из тех редких фильмов, п...",3,10 из 10
75262,458,33255,movie,2013-06-22 21:14:00,good,Тайна закрытой двери,Я имела счастье смотреть этот мультфильм в кин...,6,10 из 10


### Selecting needed columns

For baseline model we're interested only in `sentiment` and `review_body` columns

In [6]:
df = reviews_df[["sentiment", "review_body"]]

### Splitting the data

In [10]:
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["sentiment"]
)

In [11]:
train_df["sentiment"].value_counts(normalize=True)

good       0.720332
neutral    0.149974
bad        0.129694
Name: sentiment, dtype: float64

In [12]:
test_df["sentiment"].value_counts(normalize=True)

good       0.720325
neutral    0.149971
bad        0.129704
Name: sentiment, dtype: float64

## Modelling

### Tf-Idf for `review_body`

In [24]:
vectorizer_params = {
    "ngram_range": (1, 2),
    "max_features": 10000,
    "tokenizer": lambda s: s.split(),
}
vectorizer_article = TfidfVectorizer(**vectorizer_params)

In [25]:
%%timeit
X_train_review = vectorizer_article.fit_transform(train_df["review_body"])


KeyboardInterrupt



In [ ]:
%%timeit
X_test_review = vectorizer_article.transform(test_df["review_body"])

### Label Encoding

In [17]:
le = LabelEncoder()
train_labels = le.fit_transform(train_df["sentiment"])
test_labels = le.transform(test_df["sentiment"])

### Training LogReg

In [20]:
log_reg = LogisticRegression(random_state=SEED)